In [4]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from __future__ import print_function, division
from scipy import stats
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from Utility.IO import *
import math

In [7]:
def clean_and_prepare():
    # Load csv file into dataframe
    sc96 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED1996_97_PP.csv")
    sc97 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED1997_98_PP.csv")
    sc98 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED1998_99_PP.csv")
    sc99 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED1999_00_PP.csv")
    sc00 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED2000_01_PP.csv")
    sc01 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED2001_02_PP.csv")
    sc02 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED2002_03_PP.csv")
    sc03 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED2003_04_PP.csv")
    sc04 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED2004_05_PP.csv")
    sc05 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED2005_06_PP.csv")
    sc06 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED2006_07_PP.csv")
    sc07 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED2007_08_PP.csv")
    sc08 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED2008_09_PP.csv")
    sc09 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED2009_10_PP.csv")
    sc10 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED2010_11_PP.csv")
    sc11 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED2011_12_PP.csv")
    sc12 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED2012_13_PP.csv")
    sc13 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED2013_14_PP.csv")
    sc14 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED2014_15_PP.csv")
    
    # Load Dictionary
    dic_file=pd.ExcelFile("../data/CollegeScorecardDataDictionary.xlsx")
    dic_init = dic_file.parse("data_dictionary")
    dic_init.head()
    dic_init.columns
    
    # Clean Dictionary
    dic=dic_init.ix[dic_init['VARIABLE NAME'].notnull()]
    
    sc_init=[sc96,sc97,sc98,sc99,sc00,sc01,sc02,sc03,sc04,sc05,sc06,sc07,sc08,sc09,sc10,sc11,sc12,sc13,sc14]
    
    # Add year variable
    for i in range(len(sc_init)):
        sc_init[i]['Year']=i+1996
    
    # Merge dataset with valid repayment information (2007-2014)
    sc_merge=pd.concat(sc_init[11:19])
    
    # Pick x and y variables
    scx_merge=sc_merge.loc[:,dic['VARIABLE NAME'][dic["dev-category"]!="repayment"]]
    scy_merge=sc_merge.loc[:,['RPY_1YR_RT','RPY_3YR_RT','RPY_5YR_RT', 'RPY_7YR_RT'] ]
    
    # Combine x and y variables
    sc=pd.concat([scy_merge,scx_merge,sc_merge['Year']],axis=1)
    
    # Drop empty row and column
    sc_drop=sc.dropna(how="all",axis=1)
    sc_drop=sc_drop.dropna(how="all",axis=0)
    
    # Save it to local file
    sc_drop.to_csv('../data/sc_final.csv')
    
    return sc_drop

In [ ]:
# Example
sc=clean_and_prepare()
# after creating .csv file, we can obtain data from file directly
sc= pd.read_csv("../data/sc_final.csv",encoding="ISO-8859-1")

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2802: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2802: DtypeWarning: Columns (1,6,442,451,598,607,611,620,624,633,789,802,806,815,893,906,971,984,997,1153,1166,1407,1408,1411,1425,1431,1432,1433,1437,1438,1439,1440,1445,1446,1447,1451,1452,1453,1454,1459,1460,1461,1465,1466,1467,1468,1473,1474,1475,1479,1480,1481,1482,1487,1488,1489,1501,1502,1537,1538,1539,1540,1541,1542,1603,1606,1609,1610,1611,1613,1614,1615,1616,1688,1689,1690,1691,1692,1729) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2802: DtypeWarning: Columns (1,1729) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.r

In [11]:
sc.RPY_1YR_RT=pd.Series(sc.RPY_1YR_RT).convert_objects(convert_numeric=True)
sc.RPY_3YR_RT=pd.Series(sc.RPY_3YR_RT).convert_objects(convert_numeric=True)
sc.RPY_5YR_RT=pd.Series(sc.RPY_5YR_RT).convert_objects(convert_numeric=True)
sc.RPY_7YR_RT=pd.Series(sc.RPY_7YR_RT).convert_objects(convert_numeric=True)
sc.Year=sc.Year.astype('object')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  after removing the cwd from sys.path.


In [14]:
sc.RPY_1YR_RT=pd.Series(sc.RPY_1YR_RT).convert_objects(convert_numeric=True)
sc.RPY_3YR_RT=pd.Series(sc.RPY_3YR_RT).convert_objects(convert_numeric=True)
sc.RPY_5YR_RT=pd.Series(sc.RPY_5YR_RT).convert_objects(convert_numeric=True)
sc.RPY_7YR_RT=pd.Series(sc.RPY_7YR_RT).convert_objects(convert_numeric=True)
sc.FEMALE_ENRL_ORIG_YR2_RT=pd.Series(sc.FEMALE_ENRL_ORIG_YR2_RT).convert_objects(convert_numeric=True)
sc.DEP_INC_PCT_LO=pd.Series(sc.DEP_INC_PCT_LO).convert_objects(convert_numeric=True)
sc.DEP_INC_PCT_H1=pd.Series(sc.DEP_INC_PCT_H1).convert_objects(convert_numeric=True)
sc.DEP_INC_PCT_H2=pd.Series(sc.DEP_INC_PCT_H2).convert_objects(convert_numeric=True)
sc.PAR_ED_PCT_PS=pd.Series(sc.PAR_ED_PCT_PS).convert_objects(convert_numeric=True)
sc.DEP_INC_AVG=pd.Series(sc.DEP_INC_AVG).convert_objects(convert_numeric=True)
sc.PELL_EVER=pd.Series(sc.PELL_EVER).convert_objects(convert_numeric=True)
sc.PAR_ED_PCT_1STGEN=pd.Series(sc.PAR_ED_PCT_1STGEN).convert_objects(convert_numeric=True)
sc.C200_4_POOLED_SUPP=pd.Series(sc.C200_4_POOLED_SUPP).convert_objects(convert_numeric=True)
sc.Year=sc.Year.astype('object')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  after removing the cwd from sys.path.
C:\Progra

In [29]:
sc.head()

,Unnamed: 0,RPY_1YR_RT,RPY_3YR_RT,RPY_5YR_RT,RPY_7YR_RT,UNITID,OPEID,OPEID6,INSTNM,CITY,...,C100_L4,D100_L4,TRANS_4,DTRANS_4,TRANS_L4,DTRANS_L4,ICLEVEL,UGDS_MEN,UGDS_WOMEN,Year
0,0,0.4770441627,NaN,NaN,NaN,100654,100200,1002,Alabama A & M University,Normal,...,NaN,NaN,0.000000,1038.0,NaN,NaN,1.0,0.4709,0.5291,2007
1,1,0.6081853747,NaN,NaN,NaN,100663,105200,1052,University of Alabama at Birmingham,Birmingham,...,NaN,NaN,0.234477,1224.0,NaN,NaN,1.0,0.3956,0.6044,2007
2,2,0.6575342466,NaN,NaN,NaN,100690,2503400,25034,Amridge University,Montgomery,...,NaN,NaN,0.600000,5.0,NaN,NaN,1.0,0.4752,0.5248,2007
3,3,0.6572052402,NaN,NaN,NaN,100706,105500,1055,University of Alabama in Huntsville,Huntsville,...,NaN,NaN,0.331081,592.0,NaN,NaN,1.0,0.5169,0.4831,2007
4,4,0.4320479221,NaN,NaN,NaN,100724,100500,1005,Alabama State University,Montgomery,...,NaN,NaN,0.000000,1263.0,NaN,NaN,1.0,0.4026,0.5974,2007


In [2]:
def get_rpyrt_by_class(y=1):
    index={'1':1,'3':2,'5':3,'7':4}
    sc= pd.read_csv("../data/sc_final.csv",encoding="ISO-8859-1")
    yname='RPY_'+ str(y) +'YR_RT'
    sc_sub=pd.concat([sc.iloc[:,[index[str(y)]+4]],sc.drop(sc.columns[range(2,14)], axis=1)],axis=1)
    sc_sub=sc_sub.convert_objects(convert_numeric=True)
    sc_sub=sc_sub.dropna(subset=[yname])
    pro=0.3
    sc_sub=sc_sub.dropna(axis=0,how='all')
    sc_sub=sc_sub.dropna(axis=1,thresh=np.ceil(sc_sub.shape[1]*(1-pro)))
    return sc_sub

In [20]:
def get_corplot_by_class(y=1,thre=200):
    yname='RPY_'+ str(y) +'YR_RT'
    yname=math.log(1/(1-yname))
    sc_rpt=get_rpyrt_by_class(y)
    cor=sc_rpt.corr()
    corr=abs(cor).sort_values([yname],ascending=0)[yname]
    sc_cor=cor.loc[corr.index[0:thre],corr.index[0:thre]]
    fig, ax = plt.subplots(figsize=(10,7)) 
    sns.heatmap(abs(sc_cor),ax=ax, xticklabels=sc_cor.columns,yticklabels=sc_cor.columns)
    return cor.loc[corr.index[0:thre],corr.index[0:thre]]

In [3]:
sc_rpt=get_rpyrt_by_class(1)

NameError: name 'pd' is not defined

In [ ]:
sc_rpt.head()

In [17]:
sc_rpt=sc_rpt.drop(sc_rpt.columns[range(1,7)], axis=1)
cor=sc_rpt.corr()

NameError: name 'sc_rpt' is not defined

In [ ]:
corr=abs(cor).sort_values(['RPY_1YR_RT'],ascending=0)['RPY_1YR_RT']

In [ ]:
x = cor.loc[corr.index[0:200],corr.index[0:10]]
x.head()

In [ ]:
sc_sub=sc.drop(sc.columns[range(6,12)], axis=1)
sc_sub=sc_sub.convert_objects(convert_numeric=True)
sc_sub.head()

In [ ]:
sd=pd.DataFrame(np.std(sc_sub[x.index]))
new = sd[sd[0] >0.15]  
new.head()

In [ ]:
data=sc_sub[new.index]
y_var=data.RPY_1YR_RT
x_var=data.drop('RPY_1YR_RT', 1)
x_var=x_var.fillna(x_var.mean())
x_var=(x_var-x_var.mean())/x_var.std()
y_var=y_var.fillna(y_var.mean())
y_var=np.log(y_var/(1-y_var))
y_var

In [ ]:
model = LinearRegression()

In [ ]:
from sklearn.cross_validation import cross_val_score
cv = cross_val_score(model, x_var, y_var, cv=10)
cv.mean()

In [ ]:
cross_val_score(model, x_var, y_var, cv=10)

In [ ]:
np.exp(cv.mean())

In [ ]:
max(x_var)

In [ ]:
max(y_var)

In [ ]:
min(y_var)

In [ ]:
cross_val_score(model, x_var, y_var, cv=10)

In [ ]:
lm1 = smf.ols(formula='RPY_1YR_RT ~ PELL_EVER + INC_PCT_LO +  FAMINC + MD_FAMINC + DEP_INC_AVG + DEP_INC_PCT_LO + FEMALE_ENRL_ORIG_YR2_RT + MALE_ENRL_ORIG_YR2_RT + PELL_ENRL_ORIG_YR2_RT + NOT1STGEN_ENRL_ORIG_YR2_RT + NOPELL_ENRL_ORIG_YR2_RT + NOLOAN_ENRL_ORIG_YR2_RT + ENRL_ORIG_YR2_RT', data=sc_sub).fit()
lm1.summary()